# Area Depth Computation

This notebooks shows how to compute area depth from multiple polygons and one horizon.

Author: Jie Hou Modified Date: 11/30/2020

### Import the necessary library

In [3]:
from geoio.geoio import GeoIoHorizonCollection, GeoIoPolygonCollection, FileMode_update, GeoIoShapeSegment

from shapely import geometry

import math

import numpy as np

import pandas as pd

Define the input horizon collection, polygon collection and horizon name

In [6]:
HORIZON_COLLECTION = "/glb/am/sepco/proj/mex_exp2/cp_regional/ndi/salinas/events/Meijuan/Regional_Merged_Evts_FracCretPlay.int"
POLYGON_COLLECTION = "/glb/am/sepco/proj/mex_exp2/cp_regional/ndi/salinas/polygons/Team_Lead_Polygons.ply"
HORIZON_NAME = "0800_JB+RS+DT_GRID+EP"
DZ = 20

In [7]:
hc = GeoIoHorizonCollection(HORIZON_COLLECTION)

pc = GeoIoPolygonCollection(POLYGON_COLLECTION)

In [8]:
event_name = [name for name in hc.get_event_names() if HORIZON_NAME in name][0]

event = hc.get_event(event_name)

survey = hc.get_survey()

ijk_xyz_transform = survey.get_ijk_to_xyz_transform()


Read in the spill points from a csv file (Optional)

In [ ]:
#spill_point = pd.read_csv('SC2020OPT_Cret_Lead_SpillPointDepth.csv', index_col=0)

In [7]:
total_depth = survey.delta_sample()*(survey.num_sample()-1)

In [8]:
event_data = event.get_picks()

In [9]:
x = np.zeros(event_data.shape,dtype=np.float32)
y = np.zeros(event_data.shape,dtype=np.float32)

In [10]:
for i in range(event_data.shape[0]):
    for j in range(event_data.shape[1]):
        xx,yy,_ = ijk_xyz_transform.to_target((i, j, 0))
        x[i,j],y[i,j]=xx,yy

In [11]:
output = list()

Compuate Area Depth using the polygon and horizon

In [32]:
row = []

for pol_index in range(71):
    
    if (pol_index % 10 ==0):
        print(pol_index)
        
    # Read the polygon data from Polygon Collection
    polygon_name = 'SC2020OPTCret_Lead_'+str(pol_index)
    
    polygon_name = [name for name in pc.get_names() if polygon_name in name][0]
    
    polygon = pc.get_polygon(polygon_name)
    
    segments = polygon.get_segments()
    
    polygon = geometry.Polygon(segments[0].get_points())
    
    bounds = polygon.bounds
    
    # Define the working AOI for each polygon
    i1, j1, _ = ijk_xyz_transform.from_target((bounds[0], bounds[1],0))
    
    i2, j2, _ = ijk_xyz_transform.from_target((bounds[2], bounds[3],0))

    imin = math.floor(min(i1,i2))

    imax = math.ceil(max(i1,i2))

    jmin= math.floor(min(j1,j2))

    jmax = math.ceil(max(j1,j2))
   
    # Make imin,jmin to be zero if they are outside the survey
    imin = 0 if imin<0 else imin
    jmin = 0 if jmin<0 else jmin
    
    count = np.zeros(int(total_depth/DZ) + 1, dtype=np.float32)
    
    # Get the event data inside the AOI
    data = event_data[imin:imax+1, jmin: jmax+1]

    data_min = math.floor(np.nanmin(data)/DZ)*DZ
    data_max = math.floor(np.nanmax(data)/DZ)*DZ
    #data_max = math.ceil(spill_point['Spill point depth m'][pol_index]/DZ)*DZ + DZ
    num_bins = int((data_max-data_min)/DZ)
    
    index = np.zeros(data.shape,dtype = np.bool)
    
    # Compute the area depth
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
             if (geometry.Point(x[i+imin, j+jmin],y[i+imin, j+jmin]).within(polygon)):
                index[i][j]=True 
                    
    index2 = ~np.isnan(data)
    edges = np.histogram_bin_edges(data[np.logical_and(index,index2)], bins=num_bins, range=(data_min,data_max))
    edges = edges + 0.001
    his, label = np.histogram(data[np.logical_and(index,index2)], bins=edges, range=(data_min,data_max))
    
    # Convert the unti to km^2
    area = np.cumsum(his)*50*50/1000/1000
    
    for i in range(area.shape[0]):
        if (area[i]>0):
            outputarea = 0 if i ==0 else area[i-1]
            row.append({"polygon":polygon_name.split('&')[0],"depth":int(label[i]),"area":outputarea})

0
10
20
30
40
50
60
70
CPU times: user 22 s, sys: 0 ns, total: 22 s
Wall time: 22 s


Convert the output to a dataframe and write to a csv

In [33]:
output = pd.DataFrame(row)

Save output to a csv file

In [34]:
output.to_csv('~/SC2020OPT_Cret_Lead_SpillPointDepth.csv', index=False)